In [1]:
import numpy as np
import pandas as pd
import warnings
from pandas import DataFrame,Series
from scipy import stats
df=pd.read_csv('california_housing_train.csv')
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   total_bedrooms      17000 non-null  float64
 5   population          17000 non-null  float64
 6   households          17000 non-null  float64
 7   median_income       17000 non-null  float64
 8   median_house_value  17000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [3]:
from copy import deepcopy
bedrooms=df['total_bedrooms']
df = df.drop(['total_bedrooms', 'longitude', 'latitude', 'housing_median_age', 'median_income', 'median_house_value'], axis=1)
df['bedrooms'] = bedrooms
df

,total_rooms,population,households,bedrooms
0,5612.0,1015.0,472.0,1283.0
1,7650.0,1129.0,463.0,1901.0
2,720.0,333.0,117.0,174.0
3,1501.0,515.0,226.0,337.0
4,1454.0,624.0,262.0,326.0
...,...,...,...,...
16995,2217.0,907.0,369.0,394.0
16996,2349.0,1194.0,465.0,528.0
16997,2677.0,1244.0,456.0,531.0
16998,2672.0,1298.0,478.0,552.0


In [4]:
import statsmodels.api as sm
model=sm.OLS(df['bedrooms'],sm.add_constant(df[['total_rooms','population','households']]),hasconst=True)
results=model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               bedrooms   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                 1.724e+05
Date:                Thu, 08 Jun 2023   Prob (F-statistic):               0.00
Time:                        01:20:21   Log-Likelihood:                -97558.
No. Observations:               17000   AIC:                         1.951e+05
Df Residuals:                   16996   BIC:                         1.952e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.5497      0.951      1.629      0.103      -0.314       3.414
total_rooms     0.0362      0.001     53.337      0.000       0.035       0.038
population     -0.0325      0.001    -26.612      0.000      -0.035      -0.030
households      0.9748      0.005    206.806      0.000       0.966       0.984
==============================================================================
Omnibus:                    21724.653   Durbin-Watson:                   1.420
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7771448.261
Skew:                           6.813   Prob(JB):                         0.00
Kurtosis:                     106.855   Cond. No.                     6.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
results.predict(sm.add_constant(df[['total_rooms','population','households']]))

0        631.829860
1        693.119562
2        130.849378
3        259.462982
4        289.315311
            ...    
16995    412.047168
16996    501.086730
16997    502.561260
16998    522.072603
16999    304.449917
Length: 17000, dtype: float64

In [6]:
SST=((df['total_rooms']-df['total_rooms'].mean())**2).sum()
SST

80782124814.4691

In [7]:
y_hat=results.predict(sm.add_constant(df[['total_rooms','population','households']]))

In [8]:
SSE=((df['bedrooms']-y_hat)**2).sum()

In [9]:
(SST-SSE)/SST

0.9988108663742447

In [10]:
1-SSE/SST

0.9988108663742447

In [11]:
results.params

const          1.549650
total_rooms    0.036195
population    -0.032481
households     0.974831
dtype: float64

In [12]:
results.rsquared

0.9681925736223069

In [13]:
1/(1-results.rsquared)#VIF score

31.43919876212657

In [14]:
def VIF_score(df):
    VIF_score=[]
    for i in range(len(df.columns)):
        model=sm.OLS(df[df.columns[i]],sm.add_constant(df.loc[:, df.columns!=df.columns[i]]),hasconst=True)
        results=model.fit()
        VIF_score.append(1/(1-results.rsquared))
    return(VIF_score)  

In [15]:
VIF_score(df)
#A value of 1 indicates there is no correlation between a given explanatory variable and any other explanatory variables in the model.
#A value between 1 and 5 indicates moderate correlation between a given explanatory variable and other explanatory variables 
#in the model, but this is often not severe enough to require attention.
#A value greater than 5 indicates potentially severe correlation between a given explanatory variable and other explanatory
#variables in the model. In this case, the coefficient estimates and p-values in the regression output are likely unreliable.

[7.6839103990963755, 6.14950041181713, 34.74519686835205, 31.43919876212657]